#  Optimal Decisions for Discrete Stochastic Systems

We are ready to tackle our third example from this chapter's introduction: 

```{note}
In a binary communication system, 0s and 1s are transmitted over a noisy channel. If we know the probability that a given bit is a 0 and we know the probabilities associated with the noisy channel, what is the optimal decision given an observation at the output of the channel?
```

Let's start by considering one particular instantiation of this problem: 

## Binary Communication System

Suppose we are given a system with that has binary inputs and ternary outputs. Let's use $A_0$ and $A_1$ to denote the input events and $B_0$, $B_1$, and $B_2$ to denote the output events. The channel is completely specified by giving the likelihoods, $P(B_j|A_i)$ for all $i \in \{0,1\}$ and $j \in \{0,1,2\}. 

Let $p_{ij} = P(B_j|A_i)$. Note the order of the $i$ and $j$.  This is the probability of transitioning from input $i$ to output $j$, and these are also called *channel transition probabilities*. It is helpful to visualize the channel transition probabilities/likelihoods on a diagram, where we label the arrow connecting input event $A_i$ with output event $B_j$ by $p_{ij}$. An  example of such a diagram is shown in {numref}`example2to3`. For example, from this diagram, we can read that $p_{00}=5/8$, $p_{01}=1/4$, and $p_{12}=3/4$.

:::{figure-md} example2to3

<img src="example2to3.svg" alt="Channel diagram with two inputs ($A_0, A_1$) and three outputs ($B_0, B_1, B_2$). Probabilities are shown for each transition." width="400px">

Example channel transition diagram for channel with two inputs and three outputs.
:::


It is convenient to collect the channel transition probabilities into an array, such that the $(i,j)$th entry in the array is $p_{ij}$. Let's import numpy and create an array with these channel transition probabilities:

In [20]:
import numpy as np

P=np.array([
    [5/8, 1/4, 1/8],
    [1/8, 1/8, 3/4]
])

print(P)

[[0.625 0.25  0.125]
 [0.125 0.125 0.75 ]]


Note that the probabilities with the same conditioning should add to 1. In other words,

$$
\sum_{j \in \{0,1,2\} }  P(B_j|A_i) = 1, ~~~ i=0,1.
$$
In the diagram, this corresponds to the transition probabilities that emerge from the same input. In the matrix $\mathbf{P},$ this corresponds to all of the entries in a row. We can tell numpy to sum the array across the column by using the `np.sum` function and passing the `axis=1` argument to tell it to sum across the second axis (the columns):

In [22]:
np.sum(P, axis=1)

array([1., 1.])

Note that the probabilities for a particular $B_j$ (i.e., merging into a particular output) do not necessarily sum to 1.


$$
\sum_{i \in \{0,1\} }  P(B_j|A_i) = ?, ~~~ j=0,1,2.
$$

This corresponds to the column sums of the matrix $\mathbf{P}$, and we can get these column sums by using the `np.sum` function and passing the `axis=0` argument to tell numpy to sum across axis 0 (the rows):

In [23]:
np.sum(P, axis=0)

array([0.75 , 0.375, 0.875])

## Decision Problem and Optimal Solutions

The *decision problem* for the binary communication systems is concisely described as follows: given the observed output, determine which input was sent.  A *decision rule* tells how to choose an input given an observed output. In general, a decision rule may result in a randomized choice for the input,  but in this class, we will only consider deterministic decision rules.  

We can turn this into an *optimal decision problem* if we specify criteria to be optimized. Here are two common criteria:
1. Maximize the likelihood of the input 
2. Choose the input that minimizes the probability of error

The optimum decision under criterion 1 is called the *maximum likelihood (ML)* decision. We can formulate it directly using the likelihoods, which were given in {numref}`example2to3`.  For each output, we are choosing the input that has the largest likelihood, which corresponds to the arrow with the largest probability merging into that output in {numref}`example2to3`. Similarly, the ML decision corresponds to the row number with the largest probability for each column of the transition probability matrix, $\mathbf{P}$. To get the index of the largest value in each column, we can use the `np.argmax` function and pass the `axis=0` keyword parameter to tell NumPy to maximize over the rows. (Note that `np.max` would return the maximum value, whereas `np.argmax` returns the index of the maximum value.)

Thus, the ML decisions are as follows:




In [24]:
np.argmax(P, axis=0)

array([0, 0, 1])

Given $B_0$ is received decide $A_0$

Given $B_1$ is received, decide $A_0$

Given $B_2$ is received, decide $A_1$

Unfortunately, the ML solution does not necessarily minimize the probability of error. For instance, suppose that we know that 0 is sent with probability 1; i.e., $P(A_0)=1$. Then the ML rule will make an error whenever $B_2$ is received. Let's use total probability to calculate the probability of each $B_j$:

$$
P(B_j) = \sum_{i \in \{0,1\}} P(B_j|A_i) P(A_i)
$$

We start by setting up a NumPy vector to hold the *a priori* probabilities: 

$$ 
\left[ P(A_0), P(A_1) \right].
$$

Then we use a nested for loop to calculate $P(B_j)$ for each $j$ and within the loop for each $j$ we use a for loop to  carry out the sum for each $i$:

In [28]:

aprioris = np.array([1,0])
for j in range(3):
    pBj=0
    for i in range(2):
        pBj+=P[i,j]*aprioris[i]
    
    print(f'P(B{j}) = {pBj}')

P(B0) = 0.625
P(B1) = 0.25
P(B2) = 0.125


Let $E$ be the event that an error occurs (i.e., the decision differs from the transmitted symbol). Then for this simple example, $P(E) =  P(B_2) = 0.125$. We know that it is suboptimal, because we could just use the decision rule "Always decide $A_0$" and get error probability 

In [ ]:

aprioris = np.array([1/4,3/4])
aprioris = np.array([1/5,4/5])
for j in range(3):
    pBj=0
    for i in range(2):
        pBj+=P[i,j]*aprioris[i]
    print(f'P(B{j}) = {pBj}')
    for i in range(2):
        print(f'P(A{i}|B{j}) = {P[i,j]*aprioris[i]/pBj}')
    print()
        